In [1]:
import sys
sys.path.append("/home/isabrah/sustainable_high_quality_wikis")
import pandas as pd
import numpy as np
from os.path import join as opj
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from promotion_demotion_dates_utils import determine_promotion_demotion_dates
import itertools
import glob
import multiprocessing as mp
import pickle

In [2]:
def extract_page_metadata(idx, path_to_file):
    cur_article_assessment_obj = pickle.load(open(path_to_file, "rb"))
    article_id = cur_article_assessment_obj.article_id
    is_sustainable = cur_article_assessment_obj.is_sustainable
    dict_to_return = {'article_id': article_id, 'is_sustainable': is_sustainable}
    return dict_to_return

In [3]:
n_cpus = 100
usecase = 'fa' # 'fa' # 'ga'
data_folder = '/shared/3/projects/relationship-aware-networks-wikipedia/wiki_generated_data/'
output_folder = '/shared/3/projects/relationship-aware-networks-wikipedia/models'
metadata_files_folder = opj(data_folder, 'meta_data')
pickle_files_folder = opj(data_folder, 'article_assessment_objs', 'good_articles') \
    if usecase == 'ga' else opj(data_folder, 'article_assessment_objs', 'featured_articles')
# in both cases of good/featured, we have to add the 'both' folder
pickle_files_folder = [pickle_files_folder, opj(data_folder, 'article_assessment_objs', 'both')]
pickle_files_path = sorted(list(itertools.chain(*[glob.glob(opj(pff, '*.p')) for pff in pickle_files_folder])))
print(f"We have found {len(pickle_files_path)} file to process.")

We have found 7919 file to process.


In [4]:
input_for_pool = [(idx, fp) for idx, fp in enumerate(pickle_files_path)]
pool = mp.Pool(processes=n_cpus)
with pool as pool:
    results = pool.starmap(extract_page_metadata, input_for_pool)
gold_label_per_article = pd.DataFrame.from_dict(results)

In [5]:
# importing the csv with the information about topics
data_dir  = '/shared/3/projects/relationship-aware-networks-wikipedia/wikipedia_meta_data'
topics_data = pd.read_csv(opj(data_dir, 'pages_project_name_importance_only_modeled_articles.csv'))
# left join with the gold label
merged_data = topics_data.merge(right=gold_label_per_article,how="inner", left_on='page_id', right_on='article_id')
merged_data['is_sustainable_int'] = [1 if md else 0 for md in merged_data['is_sustainable']]
print(merged_data.shape)
merged_data.head()

(24324, 6)


,page_id,project_name,importance,article_id,is_sustainable,is_sustainable_int
0,25,Medicine,NaN,25,False,0
1,25,Disability,NaN,25,False,0
2,307,Military history,NaN,307,False,0
3,307,United States,top,307,False,0
4,307,U.S. Congress,mid,307,False,0


In [6]:
# group by and counting to see if we see interesting topics
per_topic_stats = merged_data.groupby('project_name').agg({'is_sustainable':'count', 'is_sustainable_int':'sum'})
per_topic_stats.columns = ['overall_cnt', 'sustainable_cnt']
per_topic_stats['sustainable_perc'] = per_topic_stats['sustainable_cnt'] / per_topic_stats['overall_cnt']
per_topic_stats.sort_values(by='sustainable_perc', ascending=False, inplace=True)

In [48]:
per_topic_stats

,overall_cnt,sustainable_cnt,sustainable_perc
project_name,,,
Rowing,13,13,1.000000
Albania,7,7,1.000000
Articles for creation,13,13,1.000000
University of Cambridge,16,16,1.000000
Snooker,40,40,1.000000
...,...,...,...
Organizations,21,10,0.476190
Cities,106,47,0.443396
Psychology,42,18,0.428571
